In [12]:
import numpy as np
import tensorflow as tf

import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# Tensorflow Basics

### Basic Definitions
- **tensor**: array of any number of dimensions of primitive values
$$ 3 $$
$$ [1, 2, 3] $$
$$ [[1, 2], [3, 4]] $$
$$ [[[1, 2, 3]], [[7, 8, 9]]] $$

- **computational graph**: series of operations arranged into a graph of nodes
    - TensorFlow core programs consist of two discrete sections
        - **Building** the computational graph
        - **Running** the computational graph

### Types of Nodes
- **constant**: no input, outputs a constant tensor
- **placeholder**: promises to provide a value, parameterizes computational graphs, used to feed in data
- **variable**: trainable variables such as weights and biases for the model, must be provided with an initial value

### Sessions
- Preface
    - Nodes do not directly evaluate to values
    - Nodes are evaluated at runtime of the computational graph
    - Computational graphs are run in sessions
- Runing Computational Graphs in Sessions
    - Encapsulates the control and state of the TensorFlow runtime

In [29]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = tf.constant(1000.0, dtype=tf.float32)
adder_node = a + b + c

session = tf.Session()
print(sess.run(adder_node, {a: [331, 7000], b: [6, 337]}))

[ 1337.  8337.]


# Linear Model Example
- **Placeholder** for input
- **Variable** for weights and bias
    - To initialize variables, you must call:
    ```
    init = tf.global_variables_initializer()
    session.run(init)
    ```

In [36]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


### Training with Loss Function
- **y** contains optimal output values
- **square_deltas** measures squared error
- **loss** is sum of squared deltas tensor

In [37]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


In [39]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


### Optimization
- **optimizers**: slowly change each variable to minimize loss function
- **gradient descent optimizer**: modifies each variable according to magnitude of the derivative of the loss with respect to that variable

In [40]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [41]:
sess.run(init) # reset variables to incorrect defaults
feed_dict = {
    x: [1, 2, 3, 4],
    y: [0, -1, -2, -3]
}

for i in range(1000):
    sess.run(train, feed_dict)

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


### Full Example

In [43]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
    sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11
